# HD-BET tutorial
[HD-BET](https://github.com/MIC-DKFZ/HD-BET) is a UNET model that performs skullstripping of pre- and post-constrast T1, T2 and FALIR MRIs.

If you are using HD-BET, please cite the following publication:

    Isensee F, Schell M, Tursunova I, Brugnara G, Bonekamp D, Neuberger U, Wick A,
    Schlemmer HP, Heiland S, Wick W, Bendszus M, Maier-Hein KH, Kickingereder P.
    Automated brain extraction of multi-sequence MRI using artificial neural
    networks. Hum Brain Mapp. 2019; 1–13. https://doi.org/10.1002/hbm.24750

### Installation
Installation instructions are available at https://github.com/MIC-DKFZ/HD-BET. Most of the time you can just run `pip install hd-bet`.

### Usage
HD-BET can be used from the terminal as described in https://github.com/MIC-DKFZ/HD-BET. 

mrid provides an API for HD-BET in `mrid.hd_bet` submodule, as well as `study.skullstrip_hd_bet`, method on `mrid.Study`. Let's review the ``mrid.hd_bet`` functionality first.

---
#### Sample dataset
For this example we download some MRIs from https://zenodo.org/records/7213153 in DICOM format.

Colin Vanden Bulcke. (2022). Open-Access DICOM MRI session (1.0) [Data set]. Zenodo. https://doi.org/10.5281/zenodo.7213153

In [ ]:
# note that since the download is large, this script may fail or download corrupted ZIP. 
# In this case you can try downloading it manually from your browser or use any other MRIs you have.

import os, shutil, requests
from pathlib import Path

if "BMAT_MRI_session" not in os.listdir("data"):

    # download
    response = requests.get("https://zenodo.org/records/7213153/files/BMAT_MRI_session.zip?download=1", stream=True)
    response.raise_for_status()
    with open("data/BMAT_MRI_session.zip", 'wb') as file:
        shutil.copyfileobj(response.raw, file) # type:ignore

    # unpack
    shutil.unpack_archive("data/BMAT_MRI_session.zip", "data/BMAT_MRI_session")

ROOT = Path("data/BMAT_MRI_session/export/home1/sdc_image_pool/images/p883/e1007")
os.listdir(ROOT)

ReadError: BMAT_MRI_session.zip is not a zip file

#### Skullstripping one image

Skullstripping one image can be performed via `mrid.hd_bet.skullstrip` function.

First thing you need to know is that HD-BET expects images to be in MNI152 space. In mrid HD-BET methods have a `register_to_mni152` argument which accepts type of modality ("T1" or "T2"), this registers input scans to MNI152 brain atlas of specified modality before passing it to HD-BET, and an inverse transform is applied to the predicted brain mask. If your input scan is not in MNI152 space or if you are not sure about this, make sure to specify the `register_to_mni152` argument with type of modality of your scan.

Then skullstripping is straightforward, pass your scan to `mrid.hd_bet.skullstrip` and it returns a `sitk.Image` of the skullstripped scan. The scans you pass can be path to a .nii.gz file, DICOM directory, sitk.Image, numpy array or torch tensor.

In [ ]:
import mrid
T1 = ROOT / "s16329"
T1_skullstripped = mrid.hd_bet.skullstrip(T1, register_to_mni152="T1")

mrid.utils.plot_study({"T1": T1, "T1_skullstripped": T1_skullstripped})

#### Getting the brain mask
If you just need the brain mask, use `mrid.hd_bet.predict_brain_mask`.

In [ ]:
brain_mask = mrid.hd_bet.skullstrip(T1, register_to_mni152="T1")

mrid.utils.plot_study({"T1": T1, "brain_mask": brain_mask, "skullstripped": mrid.mask.apply_mask(T1, brain_mask)})

#### Skullstripping multiple modalities
If you have multiple MRI modalities, it is recommended to predict brain mask on T1 (as it is the sharpest) and apply it to all modalities. In this case all modalities must be aligned. Since in the sample dataset modalities are not aligned, we will first align them using SimpleElastix. Then we can use `mrid.hd_bet.skullstrip_D` function to predict brain mask on one modality and apply it to all modalities.

`mrid.hd_bet.skullstrip_D` returns a new dictionary with the same keys as input dictionary, but all values are skull-stripped `sitk.Image`. 

if you pass `keep_original=True`, it keeps original images before skull-stripping, and add skull-stripped scans with `_hd_bet` postfix. 

You can also pass `include_mask` to add a `seg_hd_bet` key with the predicted brain mask.

In [ ]:
data = {
    "T1": ROOT / "s16329",
    "FLAIR": ROOT/"s16340",
}

# T1 and FLAIR are not aligned (they have different sizes).
# We have to first align them to some common space. 
# Since HD-BET expects MNI152, we can just align to MNI152.
# this function registers T1 to MNI152 atlas, and then registers all other modalities to T1.
data_mni152 = mrid.simple_elastix.register_each(data, key="T1", to=mrid.get_mni152("2009a T1w asymmetric"))

# run HD-BET
data_skullstripped = mrid.hd_bet.skullstrip_D(data, key="T1") # pass data["T1"] to HD-BET

print(f"{data_skullstripped = }")
mrid.utils.plot_study(data_skullstripped)

### Study interace

`study.skullstrip_hd_bet` is essentially the same as `mrid.hd_bet.skullstrip_D`. 

It does not affect segmentations (keys starting with `"seg"`). 

In [ ]:
study = mrid.Study(T1 = ROOT / "s16329", FLAIR = ROOT/"s16340")

study_mni152 = study.register_each_SE(key="T1", to=mrid.get_mni152("2009a T1w asymmetric"))

study_skullstripped = study.skullstrip_hd_bet(key="T1") # pass data["T1"] to HD-BET

print(f"{study_skullstripped = }")
study_skullstripped.plot()